<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 선언

In [ ]:
# !pip install sentencepiece transformers torch
# !pip install seqeval
# !pip install torchtext pytorch-lightning

In [1]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [4]:
from transformers import BertConfig, BertModel, AdamW

In [5]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import f1_score, accuracy_score

In [6]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

# 필요한 클래스 선언 ( 띄어쓰기 )

### KoBertTokenizer

In [7]:
# import 에러로 인해 직접 가져옴
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"


class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

### SpacingCorpusDataset

In [8]:
class SpacingCorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable[[List, List], Tuple]):
        self.sentences = []
        self.slot_labels = ["UNK", "PAD", "B", "I"]
        self.transform = transform
        self._load_data(sentences)

    def _load_data(self, sentences):
        """data를 file에서 불러온다.

        Args:
            data_path: file 경로
        """
        self.sentences = [sen.split() for sen in sentences]

    def _get_tags(self, sentence: List[str]) -> List[str]:
        """문장에 대해 띄어쓰기 tagging을 한다.
        character 단위로 분리하여 BI tagging을 한다.

        Args:
            sentence: 문장

        Retrns:
            문장의 각 토큰에 대해 tagging한 결과 리턴
            ["B", "I"]
        """

        tags = []
        for word in sentence:
            for i in range(len(word)):
                if i == 0:
                    tags.append("B")
                else:
                    tags.append("I")
        return tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = "".join(self.sentences[idx])
        sentence = [s for s in sentence]
        tags = self._get_tags(self.sentences[idx])
        tags = [self.slot_labels.index(t) for t in tags]

        (
            input_ids,
            attention_mask,
            token_type_ids,
            slot_label_ids, 
        ) = self.transform(sentence, tags)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

### Preprocessor

In [9]:
from typing import List, Tuple

class SpacingPreprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, sentence, tags
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging

        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """

        input_tokens = []
        slot_label_ids = []
					
        # tokenize
        for word, tag in zip(sentence, tags):
            tokens = self.tokenizer.tokenize(word)

            if len(tokens) == 0:
                tokens = self.tokenizer.unk_token

            input_tokens.extend(tokens)

            for i in range(len(tokens)):
                if i == 0:
                    slot_label_ids.extend([tag])
                else:
                    slot_label_ids.extend([self.pad_token_id])

        # max_len보다 길이가 길면 뒤에 자르기
        if len(input_tokens) > self.max_len - 2:
            input_tokens = input_tokens[: self.max_len - 2]
            slot_label_ids = slot_label_ids[: self.max_len - 2]

        # cls, sep 추가
        input_tokens = (
            [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
        )
        slot_label_ids = [self.pad_token_id] + slot_label_ids + [self.pad_token_id]

        # token을 id로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        # padding
        pad_len = self.max_len - len(input_tokens)
        input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
        slot_label_ids = slot_label_ids + ([self.pad_token_id] * pad_len)
        attention_mask = attention_mask + ([0] * pad_len)
        token_type_ids = token_type_ids + ([0] * pad_len)

        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        slot_label_ids = torch.tensor(slot_label_ids, dtype=torch.long)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

### Config

In [10]:
# yaml 파일 대신에 객체로 생성
class SpacingConfig():
  def __init__(self) :
    self.task= 'korean_spacing_20210101'
    self.log_path= data_path+'/logs'
    self.bert_model =  'monologg/kobert'
    self.train_data_path= data_path+'/train.txt'
    self.val_data_path= data_path+'/val.txt'
    self.test_data_path= data_path+'/test.txt'
    self.max_len= 128
    self.train_batch_size= 64
    self.eval_batch_size= 64
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()

### BertModel

In [11]:
from math import log
class SpacingBertModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = ["UNK", "PAD", "B", "I"]
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return self.linear(self.dropout(outputs[0]))

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels

        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        print("training_epoch_end")
      

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        val_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, "val_f1": val_f1, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_f1 = torch.stack([x['val_f1'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_f1':val_f1, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log("validation_epoch_end : tensorboard_logs ", tensorboard_logs)

        return {'val_acc':val_acc, 'val_loss': avg_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
        tensorboard_logs = {
            "val_loss": val_loss,
            "val_f1": val_f1,
            "val_acc" : val_acc
        }
        print("validation_end : ", tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': val_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_f1": f1, "test_acc": acc}

    def test_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        self.log("test_f1", test_f1)
        print('test_end')
        return {"pred_slot_labels" : [x["pred_slot_labels"] for x in outputs], "test_loss": test_loss, "test_f1": test_f1, "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_f1':f1, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'test_f1':f1, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        return {'pred_slot_labels':pred_slot_labels, 'gt_slot_labels': gt_slot_labels}


    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

# 필요한 클래스 선언 ( 단락 분리 )

### CorpusDataset

In [12]:
class NSPCorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable):
        self.sentences = sentences
        self.transform = transform

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        prompt = self.sentences[idx][0]
        next_sentence = self.sentences[idx][1]
        label = self.sentences[idx][2]
        (
            input_ids,
            attention_mask,
            token_type_ids,
            label, 
        ) = self.transform(prompt, next_sentence, label)

        return input_ids, attention_mask, token_type_ids, label

### Preprocessor

In [13]:
from typing import List, Tuple

class NSPPreprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, prompt, next_sentence, label
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """두 문장(prompt, next_sentence)에 대해 tokenize한 뒤 id로 변환, 두 문장을 이버붙인다.

        Args:
            prompt: 이전 문장
            next_sentence: 이어질 다음 문장
            label : 두 문장이 이어지면 0, 아니면 1

        Returns:
            feature를 리턴한다.
            input_ids : 각 토큰의 id, 2(CLS)로 시작, 3(SEP)
            attention_mask : padding은 0, 데이터가 존재하면 1
            token_type_ids : prompt 위치는 0, next_sentence 위치는 1, 1 뒤의 0은 패딩
            label : 정답이면 [1, 0], 오답이면 [0, 1]
        """
        input_ids = self.tokenizer.encode(prompt)    # 처음과 끝 표시   # convert_tokens_to_ids에 값이 1로만 나오는 오류 발생
        input_ids_next_sen = self.tokenizer.encode(next_sentence)

        if len(input_ids) + len(input_ids_next_sen) > self.max_len :        # len(input_ids) : 192, len(input_ids_next_sen) : 570~ 일 때 앞의 문장이 모두 잘리는 오류 발생 - > 수정
          if len(input_ids) > self.max_len//2 :
            input_ids = [2] + input_ids[-self.max_len//2+1:]
          if len(input_ids_next_sen) > self.max_len//2 :
            input_ids_next_sen = input_ids_next_sen[:self.max_len - len(input_ids)-1] + [3]

        token_type_ids = [0]*len(input_ids)
        token_type_ids.extend([1]*len(input_ids_next_sen))
        input_ids.extend(input_ids_next_sen)

        attention_mask = [1] * len(token_type_ids)
        pad_length = self.max_len-len(input_ids)

        input_ids.extend([0] * pad_length)
        token_type_ids.extend([0] * pad_length) # pad : 0
        attention_mask.extend([0] * pad_length) # pad : 0

        input_ids = torch.tensor(input_ids, dtype=torch.int)
        attention_mask = torch.tensor(attention_mask, dtype=torch.int)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.int)

        label = [1.0, 0.] if label == 0 else [0., 1.0]
        label = torch.tensor(label, dtype=torch.float)
        return input_ids, attention_mask, token_type_ids, label

### BERTNextSentenceModel

In [14]:
from math import log
class BertOnlyNSPHead(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        print(seq_relationship_score, len(seq_relationship_score))
        return seq_relationship_score

# NextSentencePredictorOutput
class BERTNextSentenceModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.labels_type = [0,1]
        self.pad_token_id = 0
        self.softmax = torch.nn.Softmax()
        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=2
        )
        self.model = BertForNextSentencePrediction.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.cls = BertOnlyNSPHead(config)
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.labels_type)
        )

    def forward(self,
        input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, labels=None, output_attentions=None, output_hidden_states=None,
        return_dict=None, **kwargs, ):
        """
          self.model의 결과물 :  
          
          NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
          )

        """
        logits = self.model(input_ids, token_type_ids=token_type_ids).logits
        probs = self.softmax(logits)
        return probs

    def training_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
        )
        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        acc = self._calculate_accuracy(outputs, label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, label_ids) # slot_labels : labels
        acc = self._calculate_accuracy(outputs, label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        acc = torch.stack([x['acc'] for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss, 'acc': acc}
        print('training_epoch_end : ', tensorboard_logs)
        self.log("training_epoch_end : tensorboard_logs ", tensorboard_logs)

    def validation_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        loss = self._calculate_loss(outputs, slot_label_ids)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log('val_acc', val_acc)
        self.log('val_loss', avg_loss)
        return {'val_acc':val_acc, 'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": val_loss, "val_acc" : val_acc }
        return {'val_acc':val_acc, 'val_loss': val_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_acc": acc}

    def test_end(self, outputs):
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        return {"labels" : [x["labels"] for x in outputs], "test_loss": test_loss,  "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, _ = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        return {'pred_labels':outputs}

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self):
        return DataLoader(self.dataset["pred"], batch_size=1)

    def _calculate_loss(self, outputs, labels):   # 확률에서 얼마나 떨어져있는가?
        loss = F.cross_entropy(outputs, labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):   # 0.5보다 크면 1, 아니면 0으로 labeling
        active_logits = torch.argmax(outputs, dim=1)  # 큰 값의 index(위치) 가져오기 ( 128 batch이기 때문에 )
        active_labels = torch.argmax(labels, dim=1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

### Config

In [42]:
# yaml 파일 대신에 객체로 생성
class NspConfig(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'kor_nextsentence_prediction_'
    self.log_path= data_path+'/logs'
    self.bert_model = "snunlp/KR-Medium"
    self.max_len= 512
    self.train_batch_size= 32
    self.eval_batch_size= 32
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()

# pred

## NSP Prediction

In [22]:
from transformers import BertConfig, AdamW
from transformers import BertForNextSentencePrediction
from transformers import BertTokenizer

# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import numpy as np

# from tqdm import tqdm, tqdm_notebook
# from typing import Callable, List, Tuple
# from seqeval.metrics import accuracy_score

import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [17]:
import os
content = []
label = []
for (path, dir, files) in os.walk(data_path+'/news_20'):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt' and len(filename) < 8:   # 101.txt형태
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              text = f.read()
              text = text.split('<check>')
              content.extend(text)
              label.extend([i for i in range(len(text))])     # 문장의 끝과 다음 것은 다름
len(content), len(label)

(206, 206)

In [18]:
def get_dataset(li, label) :   # [sentence], [순서 여부 label]
  i = 1
  dataset = []
  while i < len(li) :
    if label[i] == 0 :      # 안이어짐 ( 새로 시작 )
      dataset.append((li[i-1], li[i], 1))
    else :                # 이어짐 ( 정답 )
      dataset.append((li[i-1], li[i], 0))
    i += 1
  return dataset

In [19]:
# 입력값 : list, 리턴값 : pd.DataFrame
def get_dataset_to_check_next_sentence(predset) :
  config = NspConfig()
  preprocessor = NSPPreprocessor(config.max_len)
  dataset = {'pred' : NSPCorpusDataset(predset, preprocessor.get_input_features)}     # get_input_features(self, prompt, next_sentence, label
  model2 = BERTNextSentenceModel(config, dataset).load_from_checkpoint(data_path+'/news_class9x1400/nsp/checkpoints/'+ config.task + '/epoch=2_val_acc=0.979335_other_metric=0.00.ckpt', config=config, dataset=dataset )
  trainer = pl.Trainer(gpus=config.gpus)
  predictions = trainer.predict(model2, dataloaders=model2.pred_dataloader())  # dataset으로 예측
  answer = []
  for p in predictions :
    answer.append([torch.argmax(p['pred_labels']).tolist(), p['pred_labels']])
  return answer

In [30]:
predset = [con.replace('\n', '').replace('\'', '') for con in content]
predset[0]

'코로나 정점 지났다는 정부…보건소 신속검사 중단 검토치료이어 검사도 개인에 떠맡겨 정부가 코로나19 확산세가 정점을지나 하락세로 전환됐다고 밝힌 가운데보건소와 선별진료소 등에서 신속항원검사(RAT)를 중단하는 방안을 검토중인 것으로 드러나 논란이 일고 있다.재택 셀프 치료에 이어 확진 여부 판단마저 국민 개개인에게 떠넘기고 있다는지적이 나온다. 25일 질병관리청 중앙방역대책본부는 "지방자치단체 의견을 수렴한 이후 관계 부처 등과 검토하고 있다"고 밝혔다. 허종식 더불어민주당 의원에 따르면 앞서 방대본은 일부지자체에 보건소 선별진료소 및 임시선별검사소 신속항원검사 중단 관련 의견 요청 공문을 전달했다. 이는 동네병·의원에서 양성이 나오면 확진이 인정돼 보건소에서 신속항원검사를 계속시행할 필요가 없다고 정부가 판단한데 따른 것으로 보인다.'

In [31]:
torch.set_printoptions(sci_mode=False)    # 소수점으로 표현 ( e 사용 안함 )

predset = get_dataset(predset, label)
output = get_dataset_to_check_next_sentence(predset)

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cl

Predicting: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### max_len : 256, case 분석

+ 정확도 : 0.9024390243902439

In [32]:
# max_len : 256
print([o[0] for o in output])
print([i[2] for i in predset])

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,

In [33]:
ssum = 0
li_wrong = []
for o, p in zip(output, predset) :
  if o[0] == p[2] :
    ssum += 1
  else : li_wrong.append([p, o])

print(ssum / len(output))

0.9024390243902439


In [34]:
print(len(li_wrong))
for w in li_wrong :
  print("\n\ncase : ", w[0][0])
  print("case : ", w[0][1])
  print("label : ", w[0][2])    # label
  print("output : ", w[1])    # output ( output값 ( argmax ), 비율)

20


case :  국산 고기’란 말이 관가에 돌 정도다. 이 간사는 “산업부는 원래 우리 기업들의 이익을 대변하면서 규제 완화와 원전수출을 추진해온 부처였다”며 “앞으로규제 완화를 비롯해 윤석열 정부의 신성장 전략을 가장 주도적으로 이끌어달라”고 당부했다. 행시 29회 수석인 이 간사는 1986년상공부에서 공직 생활을 시작한 ‘산업부 OB’다. 2000년 산업자원부 산업정책과장을 끝으로 학계로 옮겨 현재까지KAIST 경영공학부 교수로 재직 중이다.오형주 기자
case :  월성 경제성 조작 등 권력형 비리수사 속도 낼 듯검찰이 ‘산업통상자원부 블랙리스트 의혹’ 수사에 속도를 내면서 현 정권 비리를겨냥한 수사가 전방위로 확대될 조짐을 보이고 있다. 또 다른 대표 탈원전 사건인 월성 원전 경제성 평가 조작과 청와대의 울산시장 선거 개입 의혹 등에 대한 수사도탄력을 받을지 관심이 쏠린다. 산업부는 블랙리스트 사건 외에 월성원전의 경제성 평가를 조작했다는 의혹에대해서도 검찰의 조사를 받고 있다. 사건을 맡은 대전지방검찰청 형사4부(부장검사 김영남)는 지난해 6월 백운규 전 산업부장관과 채희봉 전 청와대 산업정책비서관을 직권남용과 업무방해 혐의로, 정재훈한국수력원자력 사장을 배임과 업무방해 혐의로 재판에 넘겼다. 다른 산업부 공
label :  1
output :  [0, tensor([[    1.0000,     0.0000]])]


case :  협력단은 공식 직제에 없는 상태다. 정식 단장이 없고 인력이 제한적인 데다 직접 수사권도 없다. 직제화가 이뤄지면 ‘여의도 저승사자’로 불리다 2020년 1월폐지된 합수단이 부활하는 셈이 된다. 원수석부대변인은 “(비직제가) 특별한 직함이나 기구가 없다는 의미라 (정식 직제화로) 필요한 기능이 회복될 것으로 기대한다고 인수위원들께서 말했다”고 전했다. 윤석열 당선인은 대선 후보 시절 미공개정보 이용, 주가조작 등의 증권 범죄수사·처벌 전 과정을 개편해 제재의 실효성을 강화하겠다고 공약했다.김진성/최진석 기자
cas

#### Case 20가지

+ case :  국산 고기’란 말이 관가에 돌 정도다. 이 간사는 “산업부는 원래 우리 기업들의 이익을 대변하면서 규제 완화와 원전수출을 추진해온 부처였다”며 “앞으로규제 완화를 비롯해 윤석열 **정부**의 신성장 전략을 가장 주도적으로 이끌어달라”고 당부했다. 행시 29회 수석인 이 간사는 1986년상공부에서 공직 생활을 시작한 ‘산업부 OB’다. 2000년 **산업자원부** **산업정책과장**을 끝으로 학계로 옮겨 현재까지KAIST 경영공학부 교수로 재직 중이다.오형주 기자
+ case :  월성 경제성 조작 등 권력형 비리수사 속도 낼 듯검찰이 **‘산업통상자원부 블랙리스트 의혹’** 수사에 속도를 내면서 현 **정권** 비리를겨냥한 수사가 전방위로 확대될 조짐을 보이고 있다. 또 다른 대표 탈원전 사건인 월성 원전 경제성 평가 조작과 청와대의 울산시장 선거 개입 의혹 등에 대한 수사도탄력을 받을지 관심이 쏠린다. 산업부는 블랙리스트 사건 외에 월성원전의 경제성 평가를 조작했다는 의혹에대해서도 검찰의 조사를 받고 있다. 사건을 맡은 대전지방검찰청 형사4부(부장검사 김영남)는 지난해 6월 백운규 전 산업부장관과 채희봉 전 청와대 산업정책비서관을 직권남용과 업무방해 혐의로, 정재훈한국수력원자력 사장을 배임과 업무방해 혐의로 재판에 넘겼다. 다른 산업부 공
+ label :  1
+ output :  0
+ 원인 : "정부/정권, 산업자원부/산업통상자원부"처럼 비슷한 단어가 겹쳤기 때문에 이어지는 문맥이라고 판단한 것으로 여겨진다.

<br>

+ case :  협력단은 공식 직제에 없는 상태다. 정식 단장이 없고 인력이 제한적인 데다 직접 **수사권**도 없다. 직제화가 이뤄지면 ‘여의도 저승사자’로 불리다 2020년 1월폐지된 합수단이 부활하는 셈이 된다. 원수석부대변인은 “(비직제가) 특별한 직함이나 기구가 없다는 의미라 (정식 직제화로) 필요한 기능이 회복될 것으로 기대한다고 인수위원들께서 말했다”고 전했다. 윤석열 당선인은 대선 후보 시절 미공개정보 이용, 주가조작 등의 증권 **범죄수사·처벌** 전 과정을 개편해 제재의 실효성을 강화하겠다고 **공약**했다.김진성/최진석 기자
+ case :  3년 묵힌 탈원전 **수사** 캐비닛 열었다…檢 칼끝, 文 겨냥하나검찰, 산업부 압수수색발전사 사장 일괄 사표 의혹2019년 5월에 멈춰 있던 ‘산업통상자원부 블랙리스트 수사’의 시계가 35개월만에 다시 돌아가기 시작했다. 문재인정부의 가장 큰 **정책** 실패 중 하나면서이미 각종 민형사 소송이 제기된 탈원전 정책과 직접 맞닿아 있는 수사다. 정치권과 관가에서는 ‘하필이면 이 시점에 검찰이 움직이기 시작했는지’를 놓고갖가지 해석이 나오고 있다. (...)
+ label :  1
+ output :  0
+ 원인 : "공약/정책, 범죄수사,수사권,처벌/수사"처럼 비슷한 단어가 겹쳤다.

<br>

+ case :  (...) “과거제를 폐지하라”고 주장한다. 조선시대 사대부권력의 기본 토대를 정면 공격한 것이었다.대신 그는 교육제도를 완전히 혁파하여 군(郡), 도(道), 중앙 3 단계로 계통적인 교육을 시켜 아래에서 위로 그 덕성과 식견을 보고 추천하면서 누진적(累進的)으로 상급교육기관에 올리고 중앙을 향해 인재를 배치하는 방식을 제안했다.시문(時文)이나 경전(經典)지식을 시험보고 그걸로 평생토록 권력의 위치에 오르게하는 폐단을 철폐하고 인간 됨과 실천력, 견식의 폭과 깊이가 단계마다 점검되어 오랫동안 관찰된 이후 추천되는 공거제(貢擧制)를대안으로 내놓은 것이다.이는 행정실무와 평판, 그리고 꾸준한 학습과 노력, 현장성과 실천력 모두가 다방면으로 검증되는 시스템의 확립이라고 하겠다.
case :  그런데 이러한 반계 유형원의 시무책은 안타깝지만 당대의 현실을 움직이지는 못했다.그는 1622년에 태어나 32세때 전남 부안 우반동(愚磻洞/여기에서 ‘반계/磻溪’라는 호를 따옴)의 초야에 묻혀 20년의 세월을 거쳐1670년에 『반계수록』을 탈고했다.그후 무려 100년이 지난 1770년 영조 46년국왕의 특명으로 이 저작은 경상도 감영에서 발간된다. 그리고 인쇄본을 다섯 곳의 사고(史庫)와 홍문관(弘文館)에 보관하도록 했다는 것이다.그럼에도 『반계수록』이 조선조 후기 시무(時務)의 책(策)이 되는 것은 쉽지 않았다.성호 이익은 이렇게 탄식한다. “근세에 반계유선생이 지은 (...)
+ label :  0
+ output :  1
+ 원인 : 전체 문장이 아니라 이어지는 문장 부근으로 잘라서 보니 겹치는 단어가 부족하다. 게다가 다음 문장이 "그런데"로 시작하다보니 반전되고 새로운 이야기가 시작되었다.

<br>

+ case :  이번 화물차 통행제한으로 그동안 굴포로를횡단하던 차량과 부영로 및 원적로를 종·횡단했던 차량은 **외곽**으로 우회해야 한다.시는 신규 지정한 화물차 통행제한구역을포함해 총 91곳에 교통안전표지 설치를 27일까지 완료하고, 인천경찰청과 협력해 3월 28일부터 2주 간 홍보 및 계도기간을 거쳐 4월 11일부터 단속을 실시한다는 방침이다.김을수 시 교통정책과장은 “우리 시는 자치경찰위원회, 경찰청, 도로교통공단 등 관계기관과함께 교통안전시설 개선사업을 펼쳐 어린이 보행안전 확보를 위해 지속적으로 노력할 것”이라면서 “교통안전시설물 설치도 중요하지만 무엇보다 어린이 **교통사고** 예방을 위해 운전자들이경각심을 갖고 어린이보호구역 내 운전 시 각별히 주의해야 한다”고 강조했다. 조경욱 기자
+ case :  윤석열의 지방균형발전론… 도 북부 홀대 벗어나나?후보자 시절 “중첩 규제 완화” 공약당선 후 “모든 지역 공정 기회” 강조광역 **교통망 구축** 등 개발호재 기대윤석열 대통령 당선인은 “‘지방 시대’라는모토를 갖고 새 정부를 운영할 생각”이라며수도권 집중 해소와 **지역**균형발전을 주요 국정 과제로 삼겠다는 의지를 거듭 피력했다.윤 당선인이 추구해온 ‘지역균형발전’은 모든 지역의 발전 속도를 똑같이 하는 ‘균등 지원’이 아닌 모든 지역이 공정한 기회를 갖고스스로 발전 동력을 찾는 방식이다.지역균형발전에 관심을 기울이는 윤 당선인이 경기도의 핵심 해결 과제 중 하나인 남·북부 균형발전을 해소할 수 있을지 관심이 모아진다.윤 당선인은 지난 24일 서울 종로구 통의동인수위 사무실에서 열린 지역균형발전특위간담회에서 “지방자치와 분권, 재정 독립성,지방 산업 등 어떤 것을 선택·집중할지 스스로 결정하게 하는 지방분권과 자치 자주성에서 지방 발전의 돌파구를 찾을 것이라 생각
+ label :  1
+ output :  0
+ 원인 : "교통사고/교통망 구축, 외곽/지역" 이처럼 비슷한 단어가 반복 

<br>

+ case :  (...) **윤 당선인**은 후보 시절인 지난해 10월 경기도당을 방문해 “북한과 맞닿은 북부 지역은군사 시설이 분산돼 있어 개발이 쉽지 않다”며 “이로 인해 남부와 개발 격차가 벌어져 차별을 빚어온 것”이라고 언급했다.당시 그는 ‘규제 완화를 통한 개발’에 초점을 맞춰 “접경 지역 1곳을 밀리터리시티(군사도시)로 만들고 다른 지역은 규제를 풀어 주민들이 재산권을 행사할 수 있도록 하겠다”고 공약한 바 있다.
+ case :  文대통령-尹**당선인** 오늘 靑 회동대선 19일만에… 특별한 의제 없어문재인 대통령과 윤석열 대통령 당선인이 28일 오후 6시 청와대에서 첫 회동을 한다.박경미 청와대 대변인은 27일 오전 브리핑을열고 이같은 회동 소식을 전하며 “문 대통령과윤 당선인이 청와대 상춘재에서 만찬을 겸해만나기로 했다”고 밝혔다. (...)
+ label :  1
+ output :  0
+ 원인 : "당선인 단어가 반복

<br>

+ case :  조율됐다고 설명했다.이어 “당선인은 청와대 이철희 정무수석의 제안을 보고받자마자 흔쾌히 이 사안에 대한 지속적인, 속도감 있는 진행을 주문했다”며“코로나19로 국민이 직면한 어려움, 우크라이나 사태로 인해 국내에 미친 경제적 파장, 안보에 있어서 윤 당선인이 갖고 있는 국민의 우려를 덜기 위해서라도 엄중한 상황에서 직접국민의 걱정을 덜어주는 게 중요하다는 판단을 했다”고 부연했다.그러면서 “허심탄회하게 두 분이 만나서 협의를 진행할 거라 생각한다”고 말했다.문 대통령과 윤 당선인의 회동은 지난 3월 9일 20대 대선이 치러진 지 19일만에 이뤄지는것으로 역대 현직 대통령과 당선인 간 회동으로서는 가장 늦게 이뤄지는 것이다.이제까지 ‘최장 기록’은 1992년 노태우 당시대통령과 김영삼(YS) 당시 당선인 간 18일 만의 회동이다. 
+ case :  도, ‘김혜경 법인카드 유용 의혹’ 핵심 인물 배씨 경찰 고발배씨에 출석₩소명 요구 응하지 않아도가 발송한 질의서에도 답변 없어감사 과정, 배씨 법인카드 사적 유용법인카드 유용 의혹 경찰 조사 필요경기도가 이재명 전 경기도지사의 배우자김혜경씨를 둘러싼 ‘경기도청 법인카드 유용’의혹과 관련해 의혹의 핵심 인물인 전 도청총무과 별정직 5급 배모씨를 횡령 및 업무상배임 혐의로 고발했다.27일 도에 따르면 배씨에게 횡령과 업무상배임 혐의를 적용해 25일 경기남부경찰청에고발장을 제출했다.앞서 경기도청 비서실 별정직 7급 공무원이었던 A씨는 지난 대선 직전 김씨와 배씨가 도청 법인카드를 사적으로 유용했다고 폭로했다.A씨는 배씨의 지시를 받아 도청 법인카드로 소고기와 초밥 등을 사서 김씨의 집으로배달했고 김씨의 약도 대리 처방받았다는 것이다.이와 관련해 도는 지난달 초 감사에 착수해해당 의혹이 불거된 부서로부터 법인카드 사용 내역과 공무원 등의 진술을 받았다.도는 배씨에게도 출석과 소명을 요구했으나
label :  1
output :  0, [0.9058, 0.0942]

<br>

+ case :  노밸리를 결합한 디스플레이·ICT 클러스터 조성’ ‘고양영상밸리를 활용한 K-콘텐츠 클러스터 조성’ 등을 제안했다.다만 윤 당선인 측은 ‘경기도 분도론’과 관련해선 구체적인 입장을 내놓지 않았다. 당시 **선대위** 측은 “경기북도 설치나 분도 관련해선 뚜렷한 논의가 진행되지 않았다”고 밝힌 바 있다.더불어민주당 최경자 의원(의정부1)은 “새**정부**가 지역균형발전에 관심을 갖는 만큼 도내남·북부 균형 발전에도 영향이 있을 것으로 기대한다”면서도 “균형발전을 위해선 북부의 중첩 **규제**를 완화하고 경제가 원활하게 이뤄질수 있도록 기업 등의 유치가 필요하다”고 지적했다. 
+ case :  김포시장 예비후보 교통문제 ‘한목소리’후보 4명, 분야별 **정책** 토론회 개최“5호선 김포한강선 연장 조속히 추진국민의힘 김포시장 **선거** 예비후보 토론회참석자 선출 과정에서 공정성 시비가 불거진가운데 26일 진행된 토론회에서 예비후보 4명이 격돌을 벌였다.국민의힘 김포시 갑·을 **당협위원회**가 주관한 이날 토론회에서 유영록·김병수·곽종규·김동식 등 4명의 예비후보는 교통, 교육, 환경등 분야별 정책을 놓고 열띤 토론이 이어졌다.첫 기조연설자로 나선 유영록 예비후보는“이제는 김포의 권력을 바꿔야 할 때”라며“아직도 국회 권력을 민주당이 가지고 있는만큼 지방선거에서 반드시 승리할 수 있는후보를 선택해야 한다”라고 주장했다.이어 김병수 예비후보는 “윤석열 후보를 선택한 것에 김포시민의 한 사람으로서 자랑스럽게 생각한다”며 “앞으로 김포시 변화와 희망을 위해 온 힘을 다하겠다”고 밝혔다.김동식 예비후보는 “저는 시장 재임 시절김포한강신도시를 유치할 당시 인천 2호선은6량이 연장되어야 한다”고 말하며 자신의 미래 비전을 강조했다.마지막 기조연설에 나선 곽종규 예비후보는 “저는 김포에서 30여 년간 언론에 몸담으
+ label :  1
+ output :  0
+ 원인 : 선대위/선거, 정부,규제/정책와 같이 비슷한 단어가 두 문장에 모두 나타난다.

<br>

+ case :  논의해야 한다”고 목소리를 높였다.김동식 예비후보 역시 “김포한강선 연장을위해서는 건폐장을 받아야 한다”며 “건폐장을 받은 뒤 김포 5호선 연장을 조속히 추진할것”이라고 말했다.곽종규 후보도 “2025년이 되면 쓰레기 매립지가 종료된다”면서 “김포한강선 연장을위해서는 **서울시**가 안고 있는 쓰레기 매립장과 건폐장을 함께 논의해 시가 좀 더 유리한입장에서 협의를 진행해야 한다”고 견해를내놨다.유영록 **후보**는 “김포한강선을 빠르게 추진하려면 반드시 지방 권력을 교체해야 한다”면서 “교체된 지방정부는 중앙정부와 함께 종합적인 타당성 조사를 통해 **김포한강선** 연장을 추진할 수 있다”고 강조했다. 천용남 기자
+ case :  “**김포 교통환경** 결과 반드시 내놓을 것”윤석열 대통령후보 선대본부 산하 광역교통개선단장이었던 김병수(51) 국민의힘 김포을 당협 수석부위원장이 **김포시장** **예비후보**로 등록,본격적인 선거전에 돌입했다.선출직 도전이 처음인 김 예비후보는 이회
+ label :  1
+ output :  0
+ 원인 : 비슷한 단어 중복 ( 서울시/김포시장, 김포한강선/김포 교통환경, 후보/예비후보 ) 

<br>

+ case :  지내면서는 김포한강선(서울 5호선 연장), 인천2호선 김포연장, 한강변 철책 개방 결정, 서울~강화고속도로 등 굵직한 지역 현안을 홍전 의원과 함께 진행했다.이날 김 예비후보는 인구 50만에 접어든 김포 성장의 핵심 키가 교통에 있다고 지적했다. 그는 **국토교통부**·기획재정부·서울시·중앙당 등을 오가며 김포지역 광역철도망 계획을놓고 협상을 이어왔다.그는 “**김포한강선**, GTX-D를 비롯한 김포교통환경 전반에 반드시 결과를 만들어낼것”이라고 포부를 밝혔다. 천용남 기자
+ case :  대학 병원·어린이 전문 병원 등 유치조승현 더불어민주당 중앙당 부대변인이자대통령 직속 **국가균형발전위원회** 전문위원은 **김포 첨단산업경제 도시”**를 기치로 내세우며 오는 6월1일 치러지는 지방선거에 김포시장 후보로 출사표를 던졌다.
+ label :  1
+ output :  0
+ 원인 : 비슷한 단어 중복 ( 국토교통부/국가균형발전위원회, 김포한강선/김포 첨단산업경제)

<br>

+ case :  중앙당 부대변인, 대통령 직속 기관 전문위원 등으로 활동하며 중앙 정치도 거친 유일한 **시장 후보**로서 **GTX-D** 추진, **지하철** 5호선 김포 연장 추진 등 김포 시민의 숙원 사업을 해결하고, 김포의 첨단 산업 도시 건설을주도하겠다는 큰 공약을 내세웠다.세부 공약으로 **트램, S-BRT** 설치를 통한GTX-D와의 유기적 연동 추진, 대학 병원과어린이 전문 병원의 유치로 김포 시민의 건강한 삶 보장, 문화예술체육복합단지 조성 및 활성화로 시민의 삶의 만족도를 향상 시키도록노력하는 시장이 되겠다고 밝혔다. 천용남 기자
+ case :  ‘3구청·10**철도** 시대’ 新 남양주 약속이인화 더불어민주당 남양주시장 예비후보가 오는 30일 오전 10시 30분, 남양주시청 영석홀에서 출마를 공식 선언하는 기자회견을 갖는다.이인화 **예비후보**는 출마선언을 통해 ‘신(新)남양주 구상, 3대 비전’을 밝힐 예정이다. 구체적인 내용은 ▲일반구 체제로의 행정대전환▲구청 중심 신 중심지 조성을 통한 남양주 랜드마크 구축 ▲10개 철도 노선을 통한 한강 이북 교통의 요지로 대도약 등이다.이 예비후보는 “출마 기자회견을 통해 ‘신남양주 구상, 3대 비전’을 설명하고, ‘3구청·10철도 시대’를 여는 남양주 새시대에 대한 약
+ label :  1
+ output :  0, [0.5603, 0.4397]
+ 원인 : "시장 후보/예비 후보, GTX-D, 지하철, 트램, S-BRT/철도"와 같이 비슷한 단어 중복. 신문 한 페이지에 후보자들의 공약과 인사말이 있었다. 그래서 비슷한 단어들이 더욱 반복된다.

<br>

+ case :  속을 드리겠다”고 밝혔다.이어 “이번 시장선거는 **철도, 도로** 등 남양주 숙원사업인 교통문제를 해결할 적임자를 뽑아야 하는 선거”라면서“교통문제를 해결할 경험과 실력을 갖춘 인물로 남양주 시민 여러분들에게 다가가겠다”고강조했다.이 예비후보는 청와대 국토교통비서관실 행정관과 두 명의 국토교통부 **장관** 정책보좌관을 역임하며 국토교통 전문가로서의 입지를탄탄히 다져왔다. 또한 연세대학교 대학원에서**도시공학**을 전공하고 공학박사를 취득해 정책적 전문성에 더해 실용적 학문능력까지 갖췄다는 평가를 받고 있다. 이화우·이도환 기자
+ case :  수원 군**공항** 부지 ‘K-실리콘밸리’ 조성김상회 전 청와대 **행정관**이 지난 25일 수원팔달구 선관위를 방문해 수원시장 선거 예비후보자 등록을 마쳤다고 27일 밝혔다.김 전 행정관은 제8회 6·1 전국동시지방선거에 더불어민주당 시장 예비후보로 등록하고본격적인 선거준비에 나섰다.앞서 김 전 행정관은 지난 1월 수원 군공항부지에서 “K-실리콘밸리를 만들겠다”며 “문재인 대통령의 국정철학을 수원에서 이어가겠
+ label :  1
+ output :  0
+ 원인 : 철도,도로/공항, 장관/행정관과 같이 비슷한 단어가 반복되었다.

<br>

+ case :  다”고 밝히며 **수원시장출마**를 선언했다.이어 “이제 시작인 수원특례시의 완성을 위한적임자로서 수원시민들이 자부심을 느낄 수 있도록 하겠다”며 “사람과환경을 책임지는 도시를 만들겠다”고 전했다.김 전 행정관은 ▲수원군공항 조기이전 ▲K-실리콘밸리구축 ▲미래세대 성장플랫폼 구축 ▲시내버스 완전공영제추진 ▲MICE산업활성화 ▲돌봄**도시**, 케어링시티**수원** 등을 핵심공약으로 제시하고 있다. 이명호 기자
+ case :  “8년째 답보상태 ‘**수원** R&D 사이언스 파크’ 조성 위해 최선 다할 것”**특례시**로 거듭난 수원시의 민선8기를 이끌 수장 후보에 대한 관심이 벌써부터 뜨겁다. 염태영 전 **수원시장**의 3연임 제한으로 수원시는 무주공산인 상황이다. 제8회 전국동시지방선거 수원시장 선거가약 70여 일 후인 6월1일 치뤄진다. 경기신문이 미리여·야 예비후보자들을 만나 출마의 변을 들어봤다.수원특례시장 출마 계기가 궁금하다.수원지역 3선 도의원이자, 도의회 의장으로서 무거운 책임감을 항상 가져왔다. 정치라는게 주민과 호흡하며 주민의 바람과 열망으로해나가는 것이라고 생각한다. 수원시장 출마결심도 그 연장선상에서 하게 됐다. 저의 이력과 역량을 지역발전을 위해 최대한 활용하는것이 제 역할이라고 생각한다. 내 지역의 발전방안을 모색하는 것은 정치인으로서 마땅한자세라고 본다.수원특례시의 가장 우선적해결 과제는 무엇이라고 생각하는가.
+ label :  1
+ output :  0
+ 원인 : "수원시장출마/수원시장, 도시,수원/수원,특례시"처럼 비슷한 단어가 반복되었다.

<br>

+ case :  농협중앙회 경기지역본부는 지난 25일“2022년도 제1차 경기농협 조합장포럼 운영협의회”를 수원축협 대회의실에서 개최했다.경기농협 조합장 포럼운영협의회는 경기농협의 대표 조합장 13명으로 구성된 협의체로이날 회의에서는 관내 농협의 당면현황을 공유하고 주요사항에 대한 의사결정을 위해 마련됐다,이날 의결된 ‘산불 피해지원(안)’에 따라 경기농협에서는 올 봄 발생한 대형 산불로 피해가 발생한 강원·경북 지역복구와 이재민 생활안정에 성금 5000만원을 지원하고 자율적인성금 모금 운동도 계획했다.농협중앙회 이사인 수원농협 염규종 조합장은 “이번 산불로 큰 시름을 겪은 이재민을 위로하며 **경기농협**이 이러한 어려운 이웃들에게 조금이나마 도움이 되기를 희망한다”고 밝혔다.방기열 기자
+ case :  **중기중앙회**, 최대 30% 싸게… 모바일 기획전 실시노란우산 업체 중 상품 500개 판매신선 식품 구매객 대상 할인 혜택도중소기업중앙회는 홈앤쇼핑과 함께 소상공인 판로지원을 위한 ‘모바일 기획전’을 실시한다고 3.27 밝혔다.모바일 기획전을 통해 노란우산 회원업체 중매출 우수상품, 시즌 신상품 등 평가를 거친79개 업체, 500여개 상품이 28일부터 31일까지나흘간 홈앤쇼핑 모바일 쇼핑몰(노란우산 우
+ label :  1
+ output :  0, [0.9741, 0.0259]
+ 경기농협/중기중앙회 단어의 반복으로 인한 것으로 보여진다.

<br>

+ case :  수상품관)에서 판매된다.특히 이번 기획전은 ‘코로나19엔 건강이 최고, 봄철엔 신선식품이 최고’라는 주제로 신선식품, 건강식품 등 상품을 구매한 소비자에게기존 판매가격 대비 최대 30% 혜택(최대 20%할인 및 10% 적립)을 제공하고, 기획전에 참여한 업체들에게도 2년간 보증보험 면제 등의 혜택이 주어진다.곽범국 중기중앙회 공제전무는 “판로에 어려움을 겪는 소상공인들이 조금이나마 도움을 주고자 기획전을 준비했다”고 말했다. 오재우 기자
+ case :  용인시 “머내·기흥만세 함성 되새긴다”만세운동 ‘103주년 기념’ 행사 개최만세길 걷기·만세 퍼포먼스 등 재연용인시는 지난 26일 ‘머내만세운동’과 ‘기흥독립만세운동’ 103주년 기념 행사가 열렸다고 27일 밝혔다.이날 수지구 고기초등학교에 세워진 머내만세운동 표지석 앞에서는 ‘머내만세운동 만세길 걷기행사’가 진행됐다.지역 역사연구모임인 머내여지도가 마련한
+ label :  1
+ output :  0 [ 1.0000,  0.0000]

<br>

+ case :  행사는 독립선언서 낭독, 기흥독립만세운동표지석 제막, 헌화 후 개울번던부터 신갈천을지나 기흥역까지 만세길을 걸으며 마무리했다.백군기 시장은 이날 행사 모두에 참석해 **시민**들과 함께 만세를 외치고, 이름도 없이 독립을 외쳤던 수많은 선열들을 기렸다.백 **시장**은 “머내만세운동과 기흥독립만세운동은 용인시의 자랑스런 역사로 독립을 위해 싸웠던 선열들을 반드시 기억하고 그 뜻을 본받아야 한다”며 “독립운동가 발굴과 예우에 더욱 힘쓰겠다“고 말했다. 최정용 기자
+ case :  정윤 “지역순환 경제 구축 시민참여 늘려야”사업 추진시 시민 혜택·기여도 고려시민제안 사업에 ‘**시민**심사위’ 필요성남시가 현재 도심공동화 심화와 향후 재정도까지 낮아지면서 지역 지속가능성이 나빠질 수 밖에 없어 이에 대한 대비책이 필요하다는 지적이다.성남시의회 정윤 의원(사진)은 최근 271회 임시회 제2차 본회의 5분 발언을 통해 ‘지역순환경제 구축’에 시민참여를 늘릴 것을 주문했다.정 의원은 집행부가 진행하는 사업을 입체적으로 봐야 한다고 주장했다.그는 “지금까지 사업결과가 불러올 **시민**혜택을 기준으로 기획했다면 앞으로는 그 과정속에서 돌아가는 시민의 혜택을 고려해야 한다”며 (...)
+ label :  1
+ output :  0 [0.9743, 0.0257]
+ 원인 : 시민 단어의 반복

<br>

+ case :  최대호 안양시장은“학의천은 안양천과 함께 시민들이 즐겨 찾는 지역이다. 하천변 산책로에 대한 불편사항을 수렴해 정비 및 개선을 지속해 나가겠다”고 전했다. 장순철 기자
+ case :  포천시 “에너지 절약하면 인센티브 드려요”市, 탄소포인트제 참여자 연중 모집전기 등 감축률 따라 포인트 지급포천시는 에너지를 절약하면 인센티브를지급하는 탄소포인트제 참여자를 연중 모집한다고 27일 밝혔다.탄소포인트제란 가정, 상가, 아파트 단지등 전기, 상수도, 도시가스의 사용량을 과거 2년치와 비교하여 절감했을 경우 감축률에 따라 포인트를 부여하고, 이에 상응하는 인센티브를 제공하는 온실가스 감축프로그램이다.탄소포인트 인센티브는 연 2회(6월, 12월) 연간 5만 원까지 현금, 포천사랑상품권, 그린카드 중 한 가지를 선택해 받을 수있으며, 인센티브 외에도 탄소포인트 가입자는 NH농협은행 금리우대 최대 0.3% 혜택과 환전 수수료 할인을 받을 수 있다.참여 방법은 탄소포인트제 홈페이지
+ label :  1
+ output :  0

<br>

+ case :  (www.cpoint.or.kr)에서 회원가입을 하거나, 참여 신청서를 **읍면동** 행정복지센터 또는 친환경정책과 기후변화대응팀으로 제출하면 된다.**포천시**는 올해 읍면동 및 **유관기관**과의협력을 통해 탄소포인트제 참여 가구의 적극적인 실천을 독려하며, 민간 참여율을높일 계획이다. 문석완 기자
+ case :  3조 규모 **포천** 클라우드데이터센터 건립**신북면 심곡리** 76만㎡ 부지에 조성시, 비알지글로벌**(주)**와 **업무** 협약박윤국 시장 “4차산업 재편 초석포천시는 지난 25일 오전 포천시청 대회의실에서 비알지글로벌(주)와 포천 클라우드 데이터센터 유치를 위한 업무협약을 체결했다.협약서에는 박윤국 포천시장과 비알지글로벌(주) 김수철 대표가 각각서명했다.포천시는 지난 2019년 포천시 소흘읍과 화현면 두 곳의 부지에 네이버제2데이터센터를 유치하기 위해 경쟁에뛰어들었지만 아쉽게 유치에는 이르지못했다.시는 네이버 제2데이터센터 사업부지공모에는 성공하지 못했으나 빠르게 성장하고 있는 데이터 산업 등 미래기술산업 육성을 위해 지속적으로 노력한 결과 이와 같은성과를 거두게 되었다고 밝혔다.이번 협약에 따르면 포천시 신북면 심곡리일원에 총 76만㎡(약 23만 평) 규모의 토지에건축면적 약 32만㎡의 클라우드 데이터센터가 건립될 예정이며, 사업비는 2조 5천억 원에서 3조 원이 투입될 것으로 보인다.포천시는 최근 전철 7호선 연장, 포천-화도
+ label :  1
+ output :  0
+ 원인 : "읍면동/신북면 심곡리", "유관기관/(주),업무"와 같이 비슷한 의미의 단어가 나왔다.

<br>

+ case :  상된다”고 말했다.박윤국 포천시장은 “포천 클라우드 데이터센터 유치를 통해 시의 경쟁력이 더욱 강화될것으로 예상된다. 포천시의 주요 산업 업종이4차산업으로 재편되는 초석이 될 것이며, 지방세 확충과 양질의 일자리 창출 등 지역경제 활성화에도 크게 기여할 것으로 보인다.사업이 원활하게 추진될 수 있도록 시에서는인허가 등 행정적 지원을 아끼지 않겠다”라고말했다. 문석완 기자
+ case :  “8개 지자체 힘 모으면 안양천 국가정원 지정 가능”비대면 ‘행정협의회’ 정기총회 개최최대호 시장 "생태하천 복원에 성공"최대호 안양시장이 다시 한 번 안양천 국가정원 지정을 피력했다.안양시는 지난 24일 최 시장이 비대면 온라인으로 개최된 ‘안양천 명소화·고도화 행정협의회’ 정기총회에 참석했다고 밝혔다.‘안양천 명소화·고도화 행정협의회’는 안양천 유역에 위치한 8개 지방자치단체가 안양천 명소화·고도화 사업을 공동으로 추진하는협의체다.향후 안양천이 국가정원으로 지정되는데초점을 맞추고 있으며, 최대호 시장을 포함해광명·군포·의왕시장과 구로·금천·영등포·양천구청장 등 경기와 서울시 각 4개 지자체장들로 구성돼 있다.지난해 8월 창립총회 후 첫 열린 정기총회
+ label :  1
+ output :  0

<br>

+ case :  1471만 3700원, 보조금카드 201만 7740원 등총액 8926만 2640원이 적립됐다.이는 지난해 적립금 8349만 원보다 577만원 늘어난 것으로, 구리시는 이를 세입예산에편성하여 주민복지에 사용할 예정이다.안승남 구리시장은 “법인카드이용 활성화로 투명한 예산 집행과 더불어 적립금 수익까지일거양득의 쾌거를 이루게되었다”며, “적립금은 3월 중 우리 시 예산으로편성하여 ‘구리, 시민행복특별시’를 이루기위해 주민복지를 위해사용하겠다”고 말했다.이도환 기자
+ case :  한국마사회 ‘공공데이터 제공 평가’ 3년 연속 최고 등급한국마사회(회장 정기환)가 행정안전부에서주관하는 ‘2021년 공공데이터 제공 운영실태평가’에서 3년 연속 최고등급 우수기관으로 선정됐다고 27일 밝혔다.한국마사회는 공공데이터 추진 기반 조성,개방 데이터 활용도 제고 노력 및 실적 등에서최고 점수를 획득했다.특히 지난해 민관 합동 연구를 통해 추진한지능형 스마트 마방 등 데이터 신사업이 긍정적인 평가를 받았다. 평가 의견에 따르면 ▲공공데이터 제공 운영을 위한 교육 이행 ▲데이
+ label :  1
+ output :  0

<br>

+ case :  살아감의 목표사람을 두려워하는 자는 **신**을 두려워하고 신을 두려워하는 자는 사람을 두려워하지 않는다.그 생애가 끊임없는 승리의 연속인 사람,무한한 것과 진실한 것을 위해 세상 사람들의 칭찬 때문이 아니라, 사명 속에서 자신의 의지처를 발견하는 사람, 세상의 눈에 띄지 않고 눈에 띄려고 생각도 하지 않는 사람, 그런 사람을 존경하라. 그런 사람은 자기가 그것으로 말미암아 괴로워하리라는 것을 알고 있으면서도 세상 사람들의욕을 먹는 선행을 선택하고, 진리를 선택한것이다. 가장 높은 선은 언제나 세상의 법칙에 반대한다. (에머슨)세상 사람들이 손가락질하는 사람들 가운데서 훌륭한 인물을 찾으라.네가 마땅히 해야 할 일이라고 생각하는일을 주저 없이 행하라. 그리고 그것에 대해서 어떠한 명예도 기대하지 말라. 어리석은 인간은 이성적인 행위에 대한 비판자라는 것을 기억하라.
+ case :  역사는 자라는 것이고, 자라기 때문에 변하고, 변하는 것이기때문에 금새가 나타나는 것인데, 금새가 보이면 말씀이 옵니다.모든 시대는 제 말씀을 가졌습니다. 그 말씀을 받은 사람이 예언자입니다. 그렇기 때문에 먼저 봤다. 먼저 알았다. 먼저 말한다.혹은 대신 말한다 합니다.대신은 누구 대신입니까. 물론 **하느님** 대신입니다. 하느님은 말씀하시지만 입으로하는 말이 아니기 때문에 그것을 옮기는입이 반드시 있어야 합니다. 그것이 예언자입니다. (함석헌)사람들의 지배에서 벗어나고 싶으면 신의 지배하에 들어가라. 네가 신의 지배하에 있음을 의식한다면 사람들은 너에게 어떠한 짓도 할 수 없을 것이다.주요 출처 : 톨스토이 《인생이란 무엇인가》
+ label :  0
+ output :  1
+ 원인 : 겹치는 단어가 거의 없고, 의미가 모호하기 때문에 모델이 해석하기 힘들었을 것으로 추청된다.

### max_len ( 512 )

Config의 maxlen을 512로 수정

+  정확도 : 0.8780487804878049

In [43]:
predset = [con.replace('\n', '').replace('\'', '') for con in content]
predset = get_dataset(predset, label)
output = get_dataset_to_check_next_sentence(predset)

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cl

Predicting: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [44]:
# max_len : 512
print(output)
print([i[2] for i in predset])

[[1, tensor([[    0.0000,     1.0000]])], [1, tensor([[    0.0000,     1.0000]])], [1, tensor([[    0.0000,     1.0000]])], [1, tensor([[    0.0000,     1.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[0.5156, 0.4844]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [1, tensor([[    0.0000,     1.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [1, tensor([[    0.0000,     1.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000,     0.0000]])], [0, tensor([[    1.0000

In [45]:
ssum = 0
li_wrong = []
for o, p in zip(output, predset) :
  if o[0] == p[2] :
    ssum += 1
  else : li_wrong.append([p, o])

print(ssum / len(output))

0.8780487804878049


#### 안 이어지는 경우 

In [46]:
for w in li_wrong :
  print("\n\ncase : ", w[0][0])
  print("case : ", w[0][1])
  print("label : ", w[0][2])    # label
  print("output : ", w[1])    # output



case :  국산 고기’란 말이 관가에 돌 정도다. 이 간사는 “산업부는 원래 우리 기업들의 이익을 대변하면서 규제 완화와 원전수출을 추진해온 부처였다”며 “앞으로규제 완화를 비롯해 윤석열 정부의 신성장 전략을 가장 주도적으로 이끌어달라”고 당부했다. 행시 29회 수석인 이 간사는 1986년상공부에서 공직 생활을 시작한 ‘산업부 OB’다. 2000년 산업자원부 산업정책과장을 끝으로 학계로 옮겨 현재까지KAIST 경영공학부 교수로 재직 중이다.오형주 기자
case :  월성 경제성 조작 등 권력형 비리수사 속도 낼 듯검찰이 ‘산업통상자원부 블랙리스트 의혹’ 수사에 속도를 내면서 현 정권 비리를겨냥한 수사가 전방위로 확대될 조짐을 보이고 있다. 또 다른 대표 탈원전 사건인 월성 원전 경제성 평가 조작과 청와대의 울산시장 선거 개입 의혹 등에 대한 수사도탄력을 받을지 관심이 쏠린다. 산업부는 블랙리스트 사건 외에 월성원전의 경제성 평가를 조작했다는 의혹에대해서도 검찰의 조사를 받고 있다. 사건을 맡은 대전지방검찰청 형사4부(부장검사 김영남)는 지난해 6월 백운규 전 산업부장관과 채희봉 전 청와대 산업정책비서관을 직권남용과 업무방해 혐의로, 정재훈한국수력원자력 사장을 배임과 업무방해 혐의로 재판에 넘겼다. 다른 산업부 공
label :  1
output :  [0, tensor([[    1.0000,     0.0000]])]


case :  필요는 없다. 다만 불기소 권고가 나온 상황에서 백 전 장관을 추가로 기소하려면그만한 증거를 확보해야 하므로 9개월가량 수사가 이어지고 있다. 청와대의 울산시장 선거 개입 의혹 역시수사가 지지부진한 사건 중 하나다. 서울중앙지방검찰청 공공수사2부는 2020년 1월백원우 전 청와대 민정비서관, 송철호 울산시장, 황운하 전 울산경찰청장 등 13명을 공직선거법 위반 혐의 등으로 불구속 기소했다. 검찰은 백 전 비서관과 송 시장, 황 전 청장이 2018년 지방선거를 앞두고 김기현 전울산시장 측근 비리 첩보 생성과 경찰 이첩과정

## Spacing Prediction

In [48]:
# 띄어쓰기가 필요한지 확인하기 위해서 line의 마지막 단어와 다음 line의 첫 단어 저장
def get_words(lines) :
  words, word = [], []
  first, dot, last = -1, -1, -1
  before_line = ''
  for line in lines :
    line = line.replace('\n', '').strip()
    line = line.replace('<check>', '')
    
    first, dot, last = line.find(' '), line.rfind('.'), line.rfind(' ')
    if dot == -1 :
      word.append( (line[:first], line[last:], len(line)) )
    else :
      while len(line) > dot+2 and line[dot+2] == '%' :
        dot = line[dot+2:].find('.')
        # line = before_line + line
      if dot == -1 :
        word.append( (line[:first], line[last:], len(line)) )
      else :
        word.append( (line[:first], 'FIN', len(line)))     # 줄의 마지막 ( dot 대신에 FIN)
        words.append(word)
        word = [('SRT', line[last:].strip(), len(line))]   # . 대신에 start, 줄의 시작 
    # before_line = line

  words.append(word)
  return words

In [50]:
import os
def get_dataset(data_path) :
  dataset = []
  dataset_label = []    # file의 개수와 원문 ( 줄바꿈 없앨 필요가 있음 )
  i = 0
  for (path, dir, files) in os.walk(data_path+'/news_20'):
      for filename in files:
          if filename[-8:] == 'text.txt' : 
              
              with open("%s/%s" % (path, filename[:-9] + '.txt'), encoding="utf-8") as f:
                lines = f.readlines()
                text = ''.join(lines).replace('\n', '')
                text_list = text.split('.')
                text_list = [text + '.' for text in text_list[:-1]] + [text_list[-1]]
                words = get_words(lines)
                temp = []
                for t in text_list :
                  if '<check>' in t :
                    t = t.replace('<check>', '')
                  if t.strip() == '' :
                    continue  
                  # if is_percent is True :      # 소수점 이어주기
                  #   temp[-1] += t
                    continue
                  temp.append(t)
              with open("%s/%s" % (path, filename), encoding="utf-8") as f:
                text = f.read()
                dataset_label.append([text, words, len(dataset), len(temp)])
              
              dataset.extend(temp)
              
  return dataset, dataset_label
  

In [63]:
# 입력값 : list, 리턴값 : pd.DataFrame
def get_dataset_to_check_spacing(dataset) :
  config = SpacingConfig()
  preprocessor = SpacingPreprocessor(config.max_len)
  model2 = SpacingBertModel(config, dataset).load_from_checkpoint(data_path+'/news_class9x1400/checkpoints/'+ config.task + '/last.ckpt', config=config, dataset=dataset )
  trainer = pl.Trainer(gpus=config.gpus)
  predictions = trainer.predict(model2, dataloaders=model2.pred_dataloader(SpacingCorpusDataset(dataset, preprocessor.get_input_features)))  # dataset으로 예측

  list_string = []
  list_pred_slot = []
  for t, p_dic in zip(dataset, predictions) :
    list_pred_slot.append(p_dic['pred_slot_labels'][0])   # 띄어쓰기 예측한 값
    s = ''
    for tt, v in zip(t.replace(' ', ''), p_dic['pred_slot_labels'][0]) :
      s += ' ' + tt if v == 'B' else tt
    s = s.lstrip()
    list_string.append(s)
  print("총 lines : " ,len(dataset), len(list_pred_slot), len(list_string))
  return list_pred_slot, list_string

In [64]:
def get_is_space(pred_text_list, words) :
  pred, pred_idx, pred_len = pred_text_list[0], 0, len(pred_text_list)
  answer = []
  answer_text = []
  for word in words :
    print(f'{pred_idx} line :')

    text = word[0][1]  # SRT
    start = 0
    for w in word[1:] :

      found = pred.find(text + w[0])
      if found == -1 :
        print(f"    not found : [{text+' ' + w[0]}]",pred) # 띄어쓰기 필요한 곳 
        answer.append(1)
        answer_text.append(text + ' ' + w[0])
      else :
        print(f"    found : [{text+w[0]}]",pred)
        answer.append(0)
        answer_text.append(text + w[0])
      
      start += w[2]
      if w[1] == 'FIN':
        pred_idx += 1
        if pred_idx == pred_len :
          break
        pred = pred_text_list[pred_idx]
      
      text = w[1]
  for pred in pred_text_list[pred_idx:] :
    print(pred)
  return answer, answer_text
# 1 line의 3번째 원문은 '세계 기업인과 학자들'이다.
# 띄어쓰기가 필요한 부분만 바꿔서 넣어야 정확도가 올라갈 것이다.

In [65]:
# 예측한 띄어쓰기 결과로 재생성한 기사
# 문장의 마지막 단어와 다음 line의 첫 단어의 연결 여부를 확인하여 원문에서 그 단어만 교체하였다.
def change_space(lines, answer) :
  text = ''.join(lines)

  idx = 0
  for a in answer :
    idx = text.find('\n')
    if a == 1 :
      text = text[:idx+3].replace('\n', ' ') + text[idx+3:]
    else :
      text = text[:idx+3].replace('\n', '') + text[idx+3:]
  return text

In [136]:
def get_dataset(con) :  # con : 한 단락
    dataset, dataset_label, = [], []    # 문장 마지막과 다음 문장 첫 단어
    text = ''.join(con).replace('\n', '')
    text_list = [text + '.' for text in text.split('.')]
    text_list[-1] = text_list[-1][:-1]  # . 제거 ( .으로 나눴으니까 )

    temp = []
    for t in text_list :    # <check>는 앞에서 문단별로 나눴기 때문에 없다.
      t = t.strip()
      if t in ['', '.']:
        continue
      if len(temp)>0 and len(t) < 6 : # . 포함
        temp[-1]+= t
        continue
      if len(temp) > 0 and t[0] in '0123456789' and temp[-1][-1] == '.' :
        temp[-1]+= t
        continue
      if t[-2:] in ['자]', '기자'] :
        continue
      temp.append(t)
    dataset_label.append([text, len(dataset), len(temp)])    #  원문, 예측할 단어, 데이터셋에서의 문장 시작 위치, 문장 개수
    dataset.extend(temp)
    return dataset, dataset_label

In [137]:
pred_words, predset, pred_labels = [], [], []
for con in content :
  words = get_words(con)
  data, data_label = get_dataset(con)
  pred_words.append(words)
  predset.extend(data)
  pred_labels.append(data_label)


In [123]:
list_pred_slot_label, pred_text_list = get_dataset_to_check_spacing(predset)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

총 lines :  1132 1132 1132


In [ ]:
# not found : 띄어쓰기 필요, found : 띄어쓰기 필요없음
pred_news = []
for label in pred_labels :
  answer, answer_text = get_is_space(pred_text_list[label[2]:label[2]+label[3]], label[1])
  print(label[2], label[3])
  pred_news.append(change_space(predset[label[2]:label[2]+label[3]], answer))

In [ ]:
# not found : 띄어쓰기 필요, found : 띄어쓰기 필요없음
pred_news = []
for label in pred_labels :
  answer, answer_text = get_is_space(pred_text_list[label[2]:label[2]+label[3]], label[1])
  print(label[2], label[3])
  pred_news.append(change_space(predset[label[2]:label[2]+label[3]], answer))

0 line :
    found : [ 반기"감사위원] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    not found : [ 의문" 인수위도] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    found : [ 존중을"] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    found : [감사원이] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    found : [ 업무보고에서] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    found : [ 원하지] 감사원, 靑정권말 인사에 반기"감사위원 임명 적절한지 의문" 인수위도"당선인 뜻 존중을" 감사원이 대통령직인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인 정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다.
    found : [ 감사위원] 감사원, 靑

['감사원, 靑 정권말 인사에 반기"감사위원 임명 적절한지 의문"인수위도 "당선인 뜻 존중을" 감사원이 대통령직 인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다. 공석이 된 2명의 감사위원 임명을 놓고 신구 권력의 줄다리기가 지속되는 상황에서 감사원이 사실상 인수위 측 손을 들어줬다는 해석이 나온다. 인수위는 25일 감사원 업무보고를받는 자리에서 "정권 이양기의 감사위원 임명 제청이 감사위원회 운영의 객관성과 공정성을 훼손하는 요인이 되어선 안 된다"고 강조했다. 이에 감사원관계자는 "감사위원이 견지해야 될 고도의 정치적 중립성을 감안할 때, 원칙적으로 현시점처럼 정치적 중립성과 관련된 논란이나 의심이 있을 수 있는 상황에서는 제청권을 행사하는 것이 적절한지 의문"이라고 답했다. 문 대통령 임기 말 감사원장이 새 감사위원 임명 제청을 하는 것이 부적절하다는 입장을내비친 것이다. 감사위원은 감사원장의 제청에 따라 대통령이 임명하도록돼 있다. 감사원 측은 이날 매일경제와의 통화에서 "인용된 입장과 내용대로이해해주면 될 것 같다"고 밝혔다. 감사원 최고 의결기구인 감사위원회는 감사원장을 포함해 감사위원 7명으로 구성되는데 현재 2석이 공석이다.현 정부 임기 말 감사위원 임명을 두고청와대와 윤 당선인은 이견을 보이고있다. 청와대는 이런 감사원 태도에도달라지는 건 없다는 입장이다. 청와대관계자는 "애초에 청와대가 일방적으로 임명하겠다는 것도 아니었고 인사권을 가진 대통령이 당선인과 협의를 통해 풀겠다는 입장에는 변함이 없기 때문에 당선인 측과 논의할 것"이라고 말했다. [김대기 기자 / 임성현 기자]',
 '김정은 "미국과 장기전"…美, 북·러 동시제재金, 신형 ICBM 발사 현장 참관하며 美본토 전역 타격능력 과시文이어 尹 "北이 얻을건 없다" 韓, 스텔스기 28대 대규모 훈련 북한이 25일 신형 대륙간탄도미사일(ICBM)인 \'화성-17형\'을 전날 발사했다고 발표하면서 미국

In [ ]:
pred_news

['감사원, 靑 정권말 인사에 반기"감사위원 임명 적절한지 의문"인수위도 "당선인 뜻 존중을" 감사원이 대통령직 인수위원회 업무보고에서 윤석열 대통령 당선인이 원하지 않는 인사에 대해 문재인정부가 감사위원 임명 제청을 요구하는 건 적절치 않다고 답했다. 공석이 된 2명의 감사위원 임명을 놓고 신구 권력의 줄다리기가 지속되는 상황에서 감사원이 사실상 인수위 측 손을 들어줬다는 해석이 나온다. 인수위는 25일 감사원 업무보고를받는 자리에서 "정권 이양기의 감사위원 임명 제청이 감사위원회 운영의 객관성과 공정성을 훼손하는 요인이 되어선 안 된다"고 강조했다. 이에 감사원관계자는 "감사위원이 견지해야 될 고도의 정치적 중립성을 감안할 때, 원칙적으로 현시점처럼 정치적 중립성과 관련된 논란이나 의심이 있을 수 있는 상황에서는 제청권을 행사하는 것이 적절한지 의문"이라고 답했다. 문 대통령 임기 말 감사원장이 새 감사위원 임명 제청을 하는 것이 부적절하다는 입장을내비친 것이다. 감사위원은 감사원장의 제청에 따라 대통령이 임명하도록돼 있다. 감사원 측은 이날 매일경제와의 통화에서 "인용된 입장과 내용대로이해해주면 될 것 같다"고 밝혔다. 감사원 최고 의결기구인 감사위원회는 감사원장을 포함해 감사위원 7명으로 구성되는데 현재 2석이 공석이다.현 정부 임기 말 감사위원 임명을 두고청와대와 윤 당선인은 이견을 보이고있다. 청와대는 이런 감사원 태도에도달라지는 건 없다는 입장이다. 청와대관계자는 "애초에 청와대가 일방적으로 임명하겠다는 것도 아니었고 인사권을 가진 대통령이 당선인과 협의를 통해 풀겠다는 입장에는 변함이 없기 때문에 당선인 측과 논의할 것"이라고 말했다. [김대기 기자 / 임성현 기자]',
 '김정은 "미국과 장기전"…美, 북·러 동시제재金, 신형 ICBM 발사 현장 참관하며 美본토 전역 타격능력 과시文이어 尹 "北이 얻을건 없다" 韓, 스텔스기 28대 대규모 훈련 북한이 25일 신형 대륙간탄도미사일(ICBM)인 \'화성-17형\'을 전날 발사했다고 발표하면서 미국

In [ ]:
len(list_pred_slot_label)

408

In [ ]:
import pandas as pd

pred_dataset = pd.DataFrame({'input_data' : [data[0].replace('\n', '').strip() for data in dataset_label], 'pred_content' : pred_news })
pred_dataset.head()

,input_data,pred_content
0,"감사원, 靑 정권말 인사에 반기""감사위원 임명 적절한지 의문""인수위도 ""당선인 뜻 ...","감사원, 靑 정권말 인사에 반기""감사위원 임명 적절한지 의문""인수위도 ""당선인 뜻 ..."
1,"김정은 ""미국과 장기전""…美, 북·러 동시제재金, 신형 ICBM 발사 현장 참관하며...","김정은 ""미국과 장기전""…美, 북·러 동시제재金, 신형 ICBM 발사 현장 참관하며..."
2,"배달 시대, 한식당이 밀려난다재료비·인건비 비싼데다가양식·중식보다 배달 번거로워서울...","배달 시대, 한식당이 밀려난다재료비·인건비 비싼데다가양식·중식보다 배달 번거로워서울..."
3,서울 아파트 매수 살아나규제 완화 기대감에12개구서 가격 반등대선 이후 재건축 지역...,서울 아파트매수 살아나규제 완화 기대감에12개구서 가격 반등 대선 이후 재건축 지역...
4,"`고졸신화` 함영주, 하나금융그룹 이끈다금융지주 주주총회 개최10년만에 새로운 `하...","`고졸신화` 함영주, 하나금융그룹 이끈다금융지주 주주총회 개최10년만에 새로운 `하..."


In [ ]:
for d, p in zip(dataset_label, pred_news) :
  if p.replace('\n', ' ').strip() != d[0].replace('\n', '').strip() :
    print(p.replace('\n', ' ').strip())
    print('----')
    print(d[0].replace('\n', '').strip())
    print('-----------------------')
